# Pre Data Preprocessing - Task [002]

## Purpose

1. Splitting text components for better granularity.
2. Normalizing numerical values to ensure consistency.
3. Removing unnecessary symbols or irrelevant text.

The preprocessing will address these specific attributes in the dataset:
- **Style**
- **Characteristics**
- **Price**
- **Capacity**
- **ABV (Alcohol by Volume)**
- **Vintage**

In [1]:
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import numpy as np
import datetime
import re

# Load the dataset
file_path = '../datasets/WineDataset.csv'
df = pd.read_csv(file_path)

def convert_to_liters(capacity):
    capacity = str(capacity).strip().upper()
    if 'CL' in capacity:  # Centiliters to Liters
        return float(re.sub(r'[^\d.]', '', capacity)) / 100
    elif 'ML' in capacity:  # Milliliters to Liters
        return float(re.sub(r'[^\d.]', '', capacity)) / 1000
    elif 'LITRE' in capacity or 'L' in capacity:  # Liters already
        return float(re.sub(r'[^\d.]', '', capacity))
    elif 'LTR' in capacity or 'L' in capacity:  # Liters already
        return float(re.sub(r'[^\d.]', '', capacity))
    elif 'L' in capacity or 'L' in capacity:  # Liters already
        return float(re.sub(r'[^\d.]', '', capacity))
    else:
        return ''  # Handle any unknown format

def preprocess_data(df):

    numeric_cols = ['Price', 'ABV', 'Capacity']

    df['Capacity'] = df['Capacity'].apply(convert_to_liters)

    if not df.empty:
        for col in numeric_cols:
            if col in df.columns:
                # Remove non-numeric characters and convert to float
                df[col] = df[col].apply(lambda x: re.sub(r'[^\d.]', '', str(x)).strip() if str(x).strip() else np.nan)
                df[col] = pd.to_numeric(df[col], errors='coerce')
                
                if df[col].notnull().any():  # Check if there's valid data for scaling
                    scaler = MinMaxScaler()
                    df[col] = scaler.fit_transform(df[[col]])
                
                df[col] = df[col].round(3)

        # Clean and split the 'Style' column
        if 'Style' in df.columns:
            df['Style'] = (
                df['Style']
                .str.replace(r'[^\w\s&]', '', regex=True)
                .str.split('&')
                .apply(lambda x: [item.strip() for item in x] if isinstance(x, list) else x)  # Clean whitespace
            )

            # This code divides the 'Style' array into several columns, each representing a position in that array
            max_len = df['Style'].apply(lambda x: len(x) if isinstance(x, list) else 0).max()

            for i in range(1, max_len + 1):
                df[f'Style {i}'] = df['Style'].apply(lambda x: x[i-1] if isinstance(x, list) and len(x) >= i else '')

            df = df.drop(columns=['Style'])

        # Clean and split the 'Characteristics' column
        if 'Characteristics' in df.columns:
            df['Characteristics'] = (
                df['Characteristics']
                .str.replace(r'[^\w\s,]', '', regex=True)
                .str.split(',') 
                .apply(lambda x: [item.strip() for item in x] if isinstance(x, list) else x)  # Clean whitespace
            )
            
            # This code divides the 'Characteristics' array into several columns, each representing a position in that array
            max_len = df['Characteristics'].apply(lambda x: len(x) if isinstance(x, list) else 0).max()

            for i in range(1, max_len + 1):
                df[f'Characteristic {i}'] = df['Characteristics'].apply(lambda x: x[i-1] if isinstance(x, list) and len(x) >= i else '')

            df = df.drop(columns=['Characteristics'])
            
        # Clean and normalize the 'Vintage' column
        if 'Vintage' in df.columns:
            current_year = datetime.datetime.now().year

            df['Vintage'] = df['Vintage'].apply(
                lambda x: current_year if str(x).strip().upper() == 'NV' else (int(re.search(r'\d{4}', str(x)).group(0)) if re.search(r'\d{4}', str(x)) else np.nan)
            )

            valid_years = df['Vintage'][df['Vintage'] > 1900]
            if not valid_years.empty:

                min_year = valid_years.min()  
                max_year = current_year

                # Calculates the vintage value based on the max vintage and the current year
                df['Vintage'] = df['Vintage'].apply(
                    lambda x: max(0, (x - max_year) / (min_year - max_year)) if pd.notna(x) else np.nan
                )

                # Round the 'Vintage' values to 2 decimal places
                df['Vintage'] = df['Vintage'].round(2)

    return df

# Preprocess the dataset
df_cleaned = preprocess_data(df)

# Save or display the cleaned dataset
df_cleaned.to_csv('../datasets/cleaned_wines.csv', index=False)
df_cleaned.head()


,Title,Description,Price,Capacity,Grape,Secondary Grape Varieties,Closure,Country,Unit,Per bottle / case / each,...,Characteristic 1,Characteristic 2,Characteristic 3,Characteristic 4,Characteristic 5,Characteristic 6,Characteristic 7,Characteristic 8,Characteristic 9,Characteristic 10
0,"The Guv'nor, Spain",We asked some of our most prized winemakers wo...,0.012,0.081,Tempranillo,NaN,Natural Cork,Spain,10.5,per bottle,...,Vanilla,Blackberry,Blackcurrant,,,,,,,
1,Bread & Butter 'Winemaker's Selection' Chardon...,This really does what it says on the tin. Itâ€...,0.026,0.081,Chardonnay,NaN,Natural Cork,USA,10.1,per bottle,...,Vanilla,Almond,Coconut,Green Apple,Peach,Pineapple,Stone Fruit,,,
2,"Oyster Bay Sauvignon Blanc 2022, Marlborough",Oyster Bay has been an award-winning gold-stan...,0.018,0.081,Sauvignon Blanc,NaN,Screwcap,New Zealand,9.8,per bottle,...,Tropical Fruit,Gooseberry,Grapefruit,Grass,Green Apple,Lemon,Stone Fruit,,,
3,Louis Latour MÃ¢con-Lugny 2021/22,Weâ€™ve sold this wine for thirty years â€“ an...,0.031,0.081,Chardonnay,NaN,Natural Cork,France,10.1,per bottle,...,Peach,Apricot,Floral,Lemon,,,,,,
4,Bread & Butter 'Winemaker's Selection' Pinot N...,Bread & Butter is that thing that you can coun...,0.026,0.081,Pinot Noir,NaN,Natural Cork,USA,10.1,per bottle,...,Smoke,Black Cherry,Cedar,Raspberry,Red Fruit,,,,,


# Pre Data Preprocessing - Task [92]

## Purpose

1. Merging both updated_wines.csv that has the mean ratings, with the merged_wine_dataset that was the result of Report 2(id:65). Adding the rating of the first dataset to the second.

In [2]:
import pandas as pd

file1 = "../datasets/updated_wines.csv"
file2 = "../datasets/merged_wine_dataset.csv"

df1 = pd.read_csv(file1) 
df2 = pd.read_csv(file2) 

# Merge the datasets based on WineName and WineryName
merged_df = df2.merge(df1[['WineName', 'WineryName', 'Ratings']], on=['WineName', 'WineryName'], how='left')

# Save the new dataset
output_file = "../datasets/PLNTD_dataset.csv"
merged_df.to_csv(output_file, index=False)

print(f"PLNTD_dataset created and saved to {output_file}")

missing_ratings = merged_df[merged_df['Ratings'].isna()]

#Testing purposes
if not missing_ratings.empty:
    print("WARNING: Some rows in the dataset are missing a rating.")
    print(missing_ratings)
else:
    print("SUCCESS: All rows have a rating.")


FileNotFoundError: [Errno 2] No such file or directory: '../datasets/updated_wines.csv'